## Python complement to the Task.
# Compare SentimentAnalysis models - The Julia Programming Language
### @author : PseudoCodeNerd
## Task Description 
Use the amazon review data from Kaggle to test the efficiency of our Sentiment Analysis models that live in TextAnalysis.jl. Compare it with models in ScikitLearn and Spacy python libraries. Upload your results as an issue in the TextAnalysis package.

Some basic machine learning knowledge is useful for this task.

### Special thanks to Ayush Kaushal; an exemplary mentor without whom this task wouldn't be possible.

Find below, the `julia` part of the task. The python notebook would be attached too but would have sparse documentation.

> The process of algorithmically identifying and categorizing opinions expressed in text to determine the user’s attitude toward the subject of the document (or post).

This is how I understand it.

This would be done using a simple one layer neural network. NLTK was..let's just forget it! (BRAINHACKED)
## Importing Required Packages

In [ ]:
import numpy as np 
import pandas as pd 
import bz2
from tqdm import tqdm #for cool progress bars
import os
print(os.listdir("../input"))
#shifted to a NN because my NLTK model was lost as I suffered a blue screen of death.
#this is on kaggle
from sklearn.utils import shuffle #for shuffling data
import tensorflow as tf
from tensorflow.keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

# 1. Read & Preprocess data

In [ ]:
train = bz2.BZ2File('../input/train.ft.txt.bz2')
test = bz2.BZ2File('../input/test.ft.txt.bz2')

In [ ]:
train_lines = train.readlines()
test_lines = test.readlines()

In [ ]:
def review_process(review):
    review = review.split(' ', 1)[1][:-1].lower()
    review = re.sub('\d','0',review)
    if 'www.' in review or 'http:' in review or 'https:' in review or '.com' in review:
        review = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", review) #predefined regex, i don't  like regexes.
    return review

In [ ]:
#already decoded into parsable strings, cell deleted and cannot be undoes,
# weird Kaggle Jupyter Rules
def split_labels_rev(data):
    reviews = []
    labels = []
    for review in tqdm(data):
        if review.split(' ')[0] == '__label__1':
            label = [1,0]
        else:
            label = [0,1] 
        rev = review_process(review)       
        reviews.append(rev[:512])
        labels.append(label)
    return reviews, labels

splitting into labels and text

In [ ]:
train_rev, y_train = split_labels_rev(train_lines)
test_rev, y_test = split_labels_rev(test_lines)

we'll now randomly shuffle our data.

In [ ]:
train_rev, y_train = shuffle(train_rev, y_train)
test_rev, y_test = shuffle(test_rev, y_test)

let's convert our labels into arrays now

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

i learnt tokenizers yeahh!


In [ ]:
tokenizer = Tokenizer(num_words=8192)
#saw this implementation on a blog somewhere , above
tokenizer.fit_on_texts(train_rev)
# https://stackoverflow.com/questions/51699001/tokenizer-texts-to-sequences-keras-tokenizer-gives-almost-all-zeros

In [ ]:
train_tokens = tokenizer.texts_to_sequences(train_rev)
test_tokens = tokenizer.texts_to_sequences(test_rev)

### Getting to our neural net.
something else i learnt, a neural network can do almost anything from character rec, to sentiment analysis to classification. It's so cool and easy to build.

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
f_train = pad_sequences(train_tokens, maxlen=128, padding='post')
f_test = pad_sequences(test_tokens, maxlen=128, padding='post')

In [ ]:
#finally our model : using the cliched overutilised softmax, adam and crossentropy parameters
#8192 are the max features
model = tf.keras.Sequential([Embedding(8192, 1, input_shape=(128,)),
    Flatten(),Dense(2, activation='softmax')])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
#training for only 2 epochs and with a cross validation set of 0.2 also.
model.fit(f_train, y_train, batch_size=16384, epochs=2, validation_split=0.2)

> # Result

In [ ]:
model.evaluate(f_test, y_test)

We get accuracy as 0.87398 which is pretyy good as compared to our Julia model and considering very little prepreocessing was involved. (Also because I don't know how to do it lol).

**END OF REPORT**